<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/RandomForest/VariableSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import progressbar

import tensorflow as tf

# Chargement des données

In [ ]:
!cat /proc/cpuinfo

In [1]:
!rm *.csv
!rm *.zip

!wget --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/RandomForest/prostate.csv"
!wget --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/RandomForest/toys.csv"

rm: cannot remove '*.csv': No such file or directory
rm: cannot remove '*.zip': No such file or directory
--2021-05-01 13:47:03--  https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/RandomForest/prostate.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11270091 (11M) [text/plain]
Saving to: ‘prostate.csv’

prostate.csv        100%[===================>]  10.75M  29.7MB/s    in 0.4s    

2021-05-01 13:47:04 (29.7 MB/s) - ‘prostate.csv’ saved [11270091/11270091]

--2021-05-01 13:47:04--  https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/RandomForest/toys.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.github

Les données contiennent 102 observations de 6033 gènes. Dans cet ensemble, il y a 52 cancers sur les 102 patients :
 - 2 variables sont très importantes
 - Environ 20 variables sont de moyenne importance
 - Le reste des variables est de faible importance 

In [4]:
df_data=pd.read_csv('prostate.csv')
df_data = df_data.set_index(df_data['Unnamed: 0'])
df_data

,Unnamed: 0,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,x.11,x.12,x.13,x.14,x.15,x.16,x.17,x.18,x.19,x.20,x.21,x.22,x.23,x.24,x.25,x.26,x.27,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,x.37,x.38,x.39,...,x.5995,x.5996,x.5997,x.5998,x.5999,x.6000,x.6001,x.6002,x.6003,x.6004,x.6005,x.6006,x.6007,x.6008,x.6009,x.6010,x.6011,x.6012,x.6013,x.6014,x.6015,x.6016,x.6017,x.6018,x.6019,x.6020,x.6021,x.6022,x.6023,x.6024,x.6025,x.6026,x.6027,x.6028,x.6029,x.6030,x.6031,x.6032,x.6033,y
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,-0.930895,-0.839996,0.062508,-0.361594,-1.124427,-1.180492,-1.092276,-1.149624,-1.157503,0.219849,0.393209,1.077884,0.444124,2.859957,0.848765,0.036011,-0.258937,1.211018,-0.146476,-0.674053,-0.880423,-0.950528,-1.056642,1.647397,-0.853895,-0.594183,0.400268,2.893403,0.003993,3.303994,0.613875,-0.242718,-0.368845,-0.930963,0.568255,-0.181331,-0.877815,-0.805629,-0.863352,...,1.615978,-1.041017,3.117119,-0.939602,-0.673270,0.135611,1.052030,-0.907884,2.024066,0.341062,0.334084,-0.834425,-0.863889,-0.802401,-0.901310,-0.886464,-0.417996,1.029027,-0.722250,-0.551445,-0.694695,-0.866458,0.529181,-0.183353,-0.803078,-0.232096,0.028221,-0.814875,1.446131,0.459535,0.492456,-0.753197,0.377710,-0.845797,0.686091,-1.091779,0.347228,-0.901310,-0.252259,healthy
2,2,-0.751885,-0.848271,0.102895,2.421034,0.182211,0.639267,-0.224728,0.119039,1.694874,-0.074817,-0.937257,-0.713941,0.121541,-1.190374,2.972325,-1.221239,0.350843,-0.970212,-1.057280,1.234538,-0.889090,-0.497687,-1.067015,1.705817,-0.862305,-0.932145,-0.181453,2.855002,-0.940909,0.212226,-0.747706,-1.165461,-0.551414,-0.940119,-0.235521,-0.967472,-0.886456,-0.813571,-0.871853,...,1.578157,-1.144084,-0.946557,2.086767,-0.105181,-0.224766,-0.655103,-0.822523,1.487055,-1.052083,-1.191817,-0.177526,0.908659,1.375230,-0.910179,-0.895189,-0.821165,-0.260801,-0.729384,-0.118835,2.307111,-0.874989,1.460098,2.438525,-0.153736,0.116249,-0.844421,2.028789,-0.949596,2.485080,-0.959873,1.374652,-0.903991,1.332845,-0.574808,2.143548,0.101821,1.326812,-0.091503,healthy
3,3,-0.545781,-0.851687,-0.003043,-0.122089,0.837907,-1.195882,-0.931749,-1.164679,-0.549045,3.828663,-0.940777,-0.465956,-1.161186,0.128602,-1.197081,-1.225090,-0.778683,-0.973770,-1.060940,-1.083485,-0.892554,-0.963419,0.456399,1.117901,-0.351685,-0.112692,3.078506,3.815220,2.801711,-0.941670,-1.071221,-0.716058,-0.952711,-0.086904,-1.320633,-0.971027,-0.565028,-0.816946,-0.875297,...,2.028149,0.016550,-0.427162,-0.952375,-0.107730,0.028139,3.381751,-0.139355,0.333449,-1.055737,-0.127980,0.620002,0.038733,-0.813683,0.211859,-0.433105,-0.824550,0.721674,0.218468,-1.225196,4.198930,0.561484,-0.739079,-0.812142,-0.310626,1.210778,-0.847832,-0.040021,0.459976,0.765470,-0.869585,-0.362556,-0.907472,-0.341481,-0.577905,-0.006464,-0.920312,-0.085084,-0.704808,healthy
4,4,-1.078519,-0.159610,0.215347,-0.096277,1.050548,0.654914,1.086128,-1.127224,0.603476,-0.532284,-0.282926,-0.245790,-1.123769,0.020612,-0.304137,-1.186960,-0.745538,-0.543737,1.440728,-0.193887,-0.858137,0.291378,-0.638649,0.514295,-0.831620,-0.722628,-0.882678,4.305706,-0.909437,0.970176,-1.034810,0.165891,1.298984,-0.908655,0.226103,-0.935734,-0.855530,-0.783374,-0.588624,...,-0.334997,-0.307339,0.506359,-0.917290,1.714716,0.601887,-0.788539,-0.256938,-0.579331,-1.019498,-0.074173,-0.812158,-0.008984,0.693429,0.324809,-0.091904,-0.790893,-0.689536,0.274913,1.488192,-0.672488,0.321280,-0.409996,1.613594,-0.780825,3.004938,-0.813915,-1.078519,-0.335290,-1.159263,-0.928211,-0.909437,-0.872889,-0.823525,-1.055268,-1.069403,-0.792237,-0.879015,-0.672488,healthy
5,5,-0.994677,-0.751905,-1.163109,-1.130144,1.695673,0.878677,-1.097698,1.490272,1.314783,-1.112842,-0.838783,0.364127,-1.152059,-0.558337,-0.363392,-1.215826,-0.770376,-0.965045,1.994059,-1.074524,-0.884003,-0.954716,4.338047,-0.098179,-0.857244,1.400656,-0.880897,-0.886587,-0.935771,-0.933014,-1.062287,-1.160103,-0.944031,-0.934982,0.827329,-0.962308,

In [40]:
df_toys=pd.read_csv('toys.csv')
df_toys = df_toys.set_index(df_toys['Unnamed: 0'])
df_toys

,Unnamed: 0,x.V1,x.V2,x.V3,x.V4,x.V5,x.V6,x.V7,x.V8,x.V9,x.V10,x.V11,x.V12,x.V13,x.V14,x.V15,x.V16,x.V17,x.V18,x.V19,x.V20,x.V21,x.V22,x.V23,x.V24,x.V25,x.V26,x.V27,x.V28,x.V29,x.V30,x.V31,x.V32,x.V33,x.V34,x.V35,x.V36,x.V37,x.V38,x.V39,...,x.V162,x.V163,x.V164,x.V165,x.V166,x.V167,x.V168,x.V169,x.V170,x.V171,x.V172,x.V173,x.V174,x.V175,x.V176,x.V177,x.V178,x.V179,x.V180,x.V181,x.V182,x.V183,x.V184,x.V185,x.V186,x.V187,x.V188,x.V189,x.V190,x.V191,x.V192,x.V193,x.V194,x.V195,x.V196,x.V197,x.V198,x.V199,x.V200,y
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,-0.296743,-1.617985,-1.067409,-0.444225,-0.523051,-0.387087,-0.152861,-0.813153,1.821314,0.459206,-0.223613,1.097568,-0.850850,-0.785266,0.132274,0.482038,-1.983613,0.942889,2.596865,-0.343049,-0.380011,-2.380398,1.085157,-0.888244,-0.871203,0.633793,1.800543,0.760820,-1.638289,0.441141,1.710350,0.853786,0.200819,-0.006223,-0.951282,0.937988,-0.381694,2.425591,-0.078469,...,-0.251172,1.906951,0.384631,0.329516,1.260098,-0.129725,-1.011991,0.277233,-1.415776,1.198413,1.153426,0.725026,0.829842,0.039747,0.385574,-1.472286,-0.603823,-1.730519,0.379732,1.693862,0.780711,0.912187,1.767525,-0.430677,0.071865,1.080474,0.701343,0.064456,-0.619621,0.924595,0.370300,0.398044,-0.160588,0.043154,1.572425,0.800887,-0.990729,0.651969,0.068696,-1
2,2,0.486371,-1.226510,-1.143179,-0.636657,0.850600,-0.455772,-0.351081,-0.017519,1.347124,-1.109948,-0.445812,-0.554439,-0.311195,1.017659,0.449656,-0.361936,-0.085926,-0.504657,1.137268,1.083371,1.784430,-0.167004,2.085521,0.662175,-0.104739,0.200151,-0.839331,0.958518,-1.171351,-0.365596,0.186393,1.090350,-1.344049,-2.014388,0.410700,0.238906,1.334687,1.196768,-1.919587,...,-0.262593,0.105936,-1.200648,1.727008,-0.133634,-0.917945,-0.864354,-0.078567,0.722163,-0.676553,0.355075,1.080515,-0.124409,1.374917,0.497287,1.200954,2.012601,0.029440,0.062998,0.172547,0.598270,-2.177544,0.692202,0.744225,1.866060,-1.507063,-0.147119,-0.527542,-0.591152,-0.621535,0.573124,-2.377734,0.336318,-0.949745,-0.735897,0.585292,0.706542,-0.368315,-0.240767,-1
3,3,0.535142,1.355318,1.375397,-0.579428,1.305002,0.099686,0.361779,-2.053743,0.583147,0.421121,-0.932565,-0.900546,1.474189,0.156475,-0.746168,-0.847667,-0.168099,-1.486241,1.416818,-1.357959,-0.395958,-1.660047,-0.930309,-1.793992,-0.283678,-0.656871,-0.787498,1.411877,1.458813,-0.258919,0.494500,1.244011,0.301916,1.654792,1.317463,-0.327037,-0.250719,-0.527441,1.293183,...,-0.069306,0.296328,0.291794,1.284333,1.587168,-2.216640,0.783389,-0.414295,0.759765,-0.943134,-1.754285,0.830438,-0.479191,0.930959,1.290842,-0.244302,-1.018051,0.238724,1.090003,1.135828,-2.653731,-0.838220,0.145952,0.628750,-1.511405,1.124491,-0.027473,0.108547,1.158716,0.089219,0.421149,0.208149,2.192439,1.302854,-0.812417,0.341327,1.008698,-0.166779,-0.543828,1
4,4,-1.108412,-0.511105,-1.021268,-0.236729,-0.039741,-0.508630,0.626754,-1.702281,-0.461578,0.580026,0.321564,1.502270,2.123768,1.062642,-1.238982,-0.995330,0.193948,0.191298,-0.868633,-0.883037,-0.486731,1.184768,1.664748,1.869663,0.293137,-1.948130,-0.692916,-1.742261,2.654250,-0.497093,1.045789,1.584237,-0.416216,0.698336,1.553863,-0.614909,0.973363,1.120632,1.162803,...,-0.318418,0.836738,-0.046846,-0.736397,0.559300,0.540311,2.575727,-0.505777,-0.475962,0.083937,0.832040,0.439538,-0.225770,0.135025,0.705082,-0.774655,-0.264935,-0.338405,-0.161711,-0.574960,1.065603,-0.390303,-0.254957,0.935546,-0.921236,-1.351187,-0.754118,2.043094,-0.332596,1.696144,-0.003792,0.935080,-0.949002,-0.454278,0.014389,0.273597,0.522809,0.297845,-0.666710,-1
5,5,1.861425,0.769617,1.244290,0.584073,-0.496416,-1.205467,-1.347270,-0.231697,-0.345950,0.693925,-0.028164,0.438850,0.343773,-0.595585,0.434864,-0.588576,-0.684969,0.086054,0.009321,0.303367,0.572145,1.127946,0.513860,0.723561,0.867375,0.437553,-0.642393,-0.853743,-0.591145,-0.491037,1.374240,-0.869190,-0.122077,0.833898,0.203234,-1.635720,0.929811,0.042271,-0.688628,...,-0.930594,-0.2185

# Etape 1 : Première observation de l'importance des variables

In [10]:
X = df_toys.drop(columns='y')
X = X.drop(columns='Unnamed: 0')
X

,x.V1,x.V2,x.V3,x.V4,x.V5,x.V6,x.V7,x.V8,x.V9,x.V10,x.V11,x.V12,x.V13,x.V14,x.V15,x.V16,x.V17,x.V18,x.V19,x.V20,x.V21,x.V22,x.V23,x.V24,x.V25,x.V26,x.V27,x.V28,x.V29,x.V30,x.V31,x.V32,x.V33,x.V34,x.V35,x.V36,x.V37,x.V38,x.V39,x.V40,...,x.V161,x.V162,x.V163,x.V164,x.V165,x.V166,x.V167,x.V168,x.V169,x.V170,x.V171,x.V172,x.V173,x.V174,x.V175,x.V176,x.V177,x.V178,x.V179,x.V180,x.V181,x.V182,x.V183,x.V184,x.V185,x.V186,x.V187,x.V188,x.V189,x.V190,x.V191,x.V192,x.V193,x.V194,x.V195,x.V196,x.V197,x.V198,x.V199,x.V200
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.296743,-1.617985,-1.067409,-0.444225,-0.523051,-0.387087,-0.152861,-0.813153,1.821314,0.459206,-0.223613,1.097568,-0.850850,-0.785266,0.132274,0.482038,-1.983613,0.942889,2.596865,-0.343049,-0.380011,-2.380398,1.085157,-0.888244,-0.871203,0.633793,1.800543,0.760820,-1.638289,0.441141,1.710350,0.853786,0.200819,-0.006223,-0.951282,0.937988,-0.381694,2.425591,-0.078469,-0.430381,...,0.099359,-0.251172,1.906951,0.384631,0.329516,1.260098,-0.129725,-1.011991,0.277233,-1.415776,1.198413,1.153426,0.725026,0.829842,0.039747,0.385574,-1.472286,-0.603823,-1.730519,0.379732,1.693862,0.780711,0.912187,1.767525,-0.430677,0.071865,1.080474,0.701343,0.064456,-0.619621,0.924595,0.370300,0.398044,-0.160588,0.043154,1.572425,0.800887,-0.990729,0.651969,0.068696
2,0.486371,-1.226510,-1.143179,-0.636657,0.850600,-0.455772,-0.351081,-0.017519,1.347124,-1.109948,-0.445812,-0.554439,-0.311195,1.017659,0.449656,-0.361936,-0.085926,-0.504657,1.137268,1.083371,1.784430,-0.167004,2.085521,0.662175,-0.104739,0.200151,-0.839331,0.958518,-1.171351,-0.365596,0.186393,1.090350,-1.344049,-2.014388,0.410700,0.238906,1.334687,1.196768,-1.919587,-0.114916,...,-0.997409,-0.262593,0.105936,-1.200648,1.727008,-0.133634,-0.917945,-0.864354,-0.078567,0.722163,-0.676553,0.355075,1.080515,-0.124409,1.374917,0.497287,1.200954,2.012601,0.029440,0.062998,0.172547,0.598270,-2.177544,0.692202,0.744225,1.866060,-1.507063,-0.147119,-0.527542,-0.591152,-0.621535,0.573124,-2.377734,0.336318,-0.949745,-0.735897,0.585292,0.706542,-0.368315,-0.240767
3,0.535142,1.355318,1.375397,-0.579428,1.305002,0.099686,0.361779,-2.053743,0.583147,0.421121,-0.932565,-0.900546,1.474189,0.156475,-0.746168,-0.847667,-0.168099,-1.486241,1.416818,-1.357959,-0.395958,-1.660047,-0.930309,-1.793992,-0.283678,-0.656871,-0.787498,1.411877,1.458813,-0.258919,0.494500,1.244011,0.301916,1.654792,1.317463,-0.327037,-0.250719,-0.527441,1.293183,0.129546,...,-1.765599,-0.069306,0.296328,0.291794,1.284333,1.587168,-2.216640,0.783389,-0.414295,0.759765,-0.943134,-1.754285,0.830438,-0.479191,0.930959,1.290842,-0.244302,-1.018051,0.238724,1.090003,1.135828,-2.653731,-0.838220,0.145952,0.628750,-1.511405,1.124491,-0.027473,0.108547,1.158716,0.089219,0.421149,0.208149,2.192439,1.302854,-0.812417,0.341327,1.008698,-0.166779,-0.543828
4,-1.108412,-0.511105,-1.021268,-0.236729,-0.039741,-0.508630,0.626754,-1.702281,-0.461578,0.580026,0.321564,1.502270,2.123768,1.062642,-1.238982,-0.995330,0.193948,0.191298,-0.868633,-0.883037,-0.486731,1.184768,1.664748,1.869663,0.293137,-1.948130,-0.692916,-1.742261,2.654250,-0.497093,1.045789,1.584237,-0.416216,0.698336,1.553863,-0.614909,0.973363,1.120632,1.162803,1.299181,...,0.884217,-0.318418,0.836738,-0.046846,-0.736397,0.559300,0.540311,2.575727,-0.505777,-0.475962,0.083937,0.832040,0.439538,-0.225770,0.135025,0.705082,-0.774655,-0.264935,-0.338405,-0.161711,-0.574960,1.065603,-0.390303,-0.254957,0.935546,-0.921236,-1.351187,-0.754118,2.043094,-0.332596,1.696144,-0.003792,0.935080,-0.949002,-0.454278,0.014389,0.273597,0.522809,0.297845,-0.666710
5,1.861425,0.769617,1.244290,0.584073,-0.496416,-1.205467,-1.347270,-0.231697,-0.345950,0.693925,-0.028164,0.438850,0.343773,-0.595585,0.434864,-0.588576,-0.684969,0.086054,0.009321,0.303367,0.572145,1.127946,0.513860,0.723561,0.867375,0.437553,-0.642393,-0.853743,-0.591145,-0.491037,1.374240,-0.869190,-0.122077,0.833898,0.203234,-1.6

In [12]:
Y = df_toys['y']
Y

Unnamed: 0
1     -1
2     -1
3      1
4     -1
5      1
      ..
96     1
97     1
98    -1
99     1
100    1
Name: y, Length: 100, dtype: int64

In [13]:
# Informations sur les données
n = 100               # Nombre d'observations
p = 200               # Nombre de variables

m_try = 100

clf = RandomForestClassifier(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=m_try, n_jobs=-1)
clf.fit(X,np.asarray(Y))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=100,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=-1, oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [14]:
col_sorted_by_importance=clf.feature_importances_.argsort()
feat_imp = pd.DataFrame({'cols':X.columns[col_sorted_by_importance],'imps':clf.feature_importances_[col_sorted_by_importance]})
feat_imp

,cols,imps
0,x.V33,0.000049
1,x.V16,0.000114
2,x.V149,0.000151
3,x.V93,0.000154
4,x.V134,0.000156
...,...,...
195,x.V1,0.070360
196,x.V5,0.081761
197,x.V6,0.098180
198,x.V2,0.235131


In [15]:
!pip install plotly_express --upgrade -q

In [16]:
import plotly_express as px
import plotly.offline as po

px.bar(feat_imp.sort_values(['imps'], ascending=False)[:25], x='cols', y='imps', labels={'cols':'column', 'imps':'feature importance'})

# Etape 2 : Classement des variables

On commence par faire tourner plusieurs fois le random forest et on sauvegarde les importances à chaque itération :

In [168]:
# Informations sur les données
n = 100               # Nombre d'observations
p = 200               # Nombre de variables
m_try = 100
n_arbres = 2000

n_step = 1

importances = []

bar=progressbar.ProgressBar(max_value=n_step)

for i in range(n_step):
  bar.update(i+1)
  clf = RandomForestClassifier(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=m_try, n_jobs=-1)
  clf.fit(X,np.asarray(Y))
#  arr = []
#  for i in range(0,len(clf.feature_importances_)):
#    arr.append(np.array(clf.feature_importances_)[0:i].sum())
#  importances.append(clf.feature_importances_*arr)

N/A% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

In [171]:
arr = []
for i in range(0,len(clf.feature_importances_)):
  arr.append(np.array(clf.feature_importances_)[0:i].sum())


In [173]:
clf.feature_importances_*arr

array([0.00000000e+00, 1.54957615e-02, 1.03646595e-01, 1.60241099e-02,
       5.26474408e-02, 7.52061978e-02, 4.13962193e-04, 4.05228669e-04,
       6.07307690e-04, 1.81944001e-04, 3.29018187e-04, 3.73597300e-04,
       1.48871371e-04, 1.58118033e-04, 2.41037008e-04, 7.93860595e-05,
       2.16663193e-03, 5.15703794e-04, 1.90982525e-04, 4.26419241e-04,
       1.35893769e-03, 2.91467966e-04, 1.67827090e-04, 2.26340246e-04,
       8.21456980e-04, 4.27578012e-04, 4.45962750e-04, 2.29742933e-04,
       7.44119570e-04, 1.58583228e-04, 6.38081298e-04, 4.73266566e-04,
       1.85446961e-04, 3.87267104e-04, 1.38518040e-04, 4.23118847e-04,
       2.90343283e-03, 1.27242179e-03, 2.69831289e-04, 4.50008109e-04,
       2.19211314e-04, 2.44428687e-04, 1.10968299e-03, 1.67770992e-03,
       4.22899823e-04, 1.25006751e-03, 4.08447504e-04, 3.31368706e-04,
       6.92106001e-04, 1.07542451e-03, 4.87445554e-04, 2.15884530e-03,
       7.83531367e-04, 8.95678637e-04, 1.11311484e-04, 3.12896327e-04,
      

On calcul la moyenne et l'écart type de l'importance de chaque variable :

In [156]:
std = np.std(importances,0)
mean = np.mean(importances,0)

In [158]:
arr = []
for i in range(0,len(mean)):
  arr.append(np.array(mean)[0:i].sum())

In [ ]:
arr*mean

In [174]:
df_classement = pd.DataFrame(data={'mean':arr*mean,'std':std}).sort_values(by=['mean'],ascending=False)
df_classement['#var'] = df_classement.index
df_classement = df_classement.reset_index(drop=True)
df_classement[0:5]

,mean,std,#var
0,0.103647,0.007252,2
1,0.075206,0.002587,5
2,0.052647,0.003101,4
3,0.016024,0.001565,3
4,0.015496,0.006921,1


In [163]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_classement.index, y=df_classement['mean'],line=dict(color='blue', width=1)))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

# Random Forest - Choix des paramètres

La caractérisation de l'importance des variables est cruciale. Cette caractéristique va dépendre :
 - Du ratio entre le nombre de variables p et le nombre d'observations n
 - Des paramètres définissant le modèle :
    - Nombre d'arbres construits par l'algorithme (ntree)
    - Nombre de variables testées à chaque division (mtry) 
 - De la présence de variables fortements corrélées

**1. Choix du nombre d'arbres**

On choisit le nombre d'arbres en prenant comme valeur de mtry la racine carrée du nombre de variables, puis on teste plusieurs valeurs en évaluant comment réduit l'OOB en fonction du nombre d'arbres générés.  
On choisit la valeur qui stabilise le minimum de l'erreur.

In [ ]:
X = df_data.drop(columns='y')
X = X.drop(columns='Unnamed: 0')
X

In [ ]:
Y = df_data['y'].apply(lambda x: 1 if (x=='cancer') else 0)
Y

RandomForestClassifier :
  - n_estimators : Nombre d'arbre dans la forêt (ntree)
  - oob_score : Sauvegarde du score OOB
  - max_samples : Nombre maximal d'observations à utiliser
  - max_features : Nombre maximum de divisions (mtry)

In [ ]:
# Informations sur les données
n = 102               # Nombre d'observations
p = 6033              # Nombre de variables

n_arbres_max = 5000

n_arbres = np.linspace(1,n_arbres_max,10).astype(np.int32)
mtry = np.sqrt(p).astype(np.int32)
OOB_err = []

for i in n_arbres:
  print("#Arbres : %d" %i)
  clf = RandomForestClassifier(n_estimators=i, bootstrap=True, oob_score=True, max_samples = n, max_features = mtry, n_jobs=-1)
  clf.fit(X,np.asarray(Y))
  OOB_err.append(1 - clf.oob_score_)

In [ ]:
plt.plot(n_arbres,OOB_err)

**2. Choix de mtry**

Pour choisir la valeur de mtry, on fait tourner Random Forest 10 fois avec dix valeurs de mtry différentes, puis on choisit celle pour laquelle le mtry est minimal et se stabilise.

In [ ]:
# Informations sur les données
n = 102               # Nombre d'observations
p = 6033              # Nombre de variables

n_arbres = 2000
mtry_0 = (np.sqrt(p)/2).astype(np.int32)

m_try = np.linspace(mtry_0,2000,10).astype(np.int32)

OOB_err = []

for i in m_try:
   print("mtry = %s" %i)
   clf = RandomForestClassifier(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=i, n_jobs=-1)
   clf.fit(X,np.asarray(Y))
   OOB_err.append(1 - clf.oob_score_)

In [ ]:
plt.plot(m_try,OOB_err)

On choisit m_try = 1000

# Random Forest - Importance des variables

Permutation importance is a technique where we shuffle the values of a single column and run the model to see how the scores get affected. If the scores are affected greatly, then the feature is highly important to the model and if not, it does not add significant value to the model.

**1. Calcul de l'importance des variables avec la méthode Gini**

In [ ]:
from sklearn.inspection import permutation_importance

# Informations sur les données
n = 102               # Nombre d'observations
p = 6033              # Nombre de variables
n_arbres = 2000
m_try = 2011

clf = RandomForestClassifier(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=m_try, n_jobs=-1)
clf.fit(X,np.asarray(Y))

In [ ]:
col_sorted_by_importance=clf.feature_importances_.argsort()
feat_imp = pd.DataFrame({'cols':X.columns[col_sorted_by_importance],'imps':clf.feature_importances_[col_sorted_by_importance]})
feat_imp

In [ ]:
!pip install plotly_express --upgrade -q

In [ ]:
import plotly_express as px
import plotly.offline as po

px.bar(feat_imp.sort_values(['imps'], ascending=False)[:25], x='cols', y='imps', labels={'cols':'column', 'imps':'feature importance'})

**2. Calcul de l'importance des variables avec la méthode des permutations**

In [ ]:
import random

def PermImportance(X, y, clf, metric, num_iterations=100):
    '''
    Calculates the permutation importance of features in a dataset.
    Inputs:
    X: dataframe with all the features
    y: array-like sequence of labels
    clf: sklearn classifier, already trained on training data
    metric: sklearn metric, such as accuracy_score, precision_score or recall_score
    num_iterations: no. of repetitive runs of the permutation
    Outputs:
    baseline: the baseline metric without any of the columns permutated
    scores: differences in baseline metric caused by permutation of each feature, dict in the format {feature:[diffs]}
    '''
    bar=progressbar.ProgressBar(max_value=len(X.columns))
    baseline_metric=metric(y, clf.predict(X))
    scores={c:[] for c in X.columns}
    for c in X.columns:
        X1=X.copy(deep=True)
        for _ in range(num_iterations):
            temp=X1[c].tolist()
            random.shuffle(temp)
            X1[c]=temp
            score=metric(y, clf.predict(X1))
            scores[c].append(baseline_metric-score)
        bar.update(X.columns.tolist().index(c))
    return baseline_metric, scores

In [ ]:
from sklearn.inspection import permutation_importance

# Informations sur les données
n = 102               # Nombre d'observations
p = 6033              # Nombre de variables
n_arbres = 2000
m_try = 2011

clf = RandomForestClassifier(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=m_try, n_jobs=-1)
clf.fit(X,np.asarray(Y))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import progressbar

baseline, scores=PermImportance(X, np.asarray(Y), clf, recall_score, num_iterations=10)

In [ ]:
X.transpose().index

In [ ]:
fig, ax = plt.subplots()
foret_importances[3:20].plot.bar(ax=ax)
ax.set_title("Importances")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()